In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import *

In [2]:
MISSING = -9999

In [3]:
df = pd.read_csv("../data/processed/data_encoded.csv")

In [4]:
df.shape

(39325, 1566)

In [5]:
df_train = df[df["test"] == 0].drop(columns=["test"])
df_test = df[df["test"] == 1].drop(columns=["test"])

In [6]:
X = df_train.drop(columns=["id", "satisfied"]).values
y = df_train["satisfied"].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1001)

In [8]:
X_final = df_test.drop(columns=["id", "satisfied"]).values

In [9]:
X_train.shape

(21056, 1563)

In [10]:
X_test.shape

(9024, 1563)

In [11]:
X_final.shape

(9245, 1563)

In [64]:
params = {
#     'subsample': 1.0,
#     'scale_pos_weight': 0.5,
    'max_depth': 4,
#     'colsample_bytree': 1.0,
#     'colsample_bynode': 0.5,
#     'colsample_bylevel': 1.0
}

In [65]:
final_model = xgb.XGBClassifier(
    learning_rate=0.1,
    objective='binary:logistic',
    silent=True, 
    missing=MISSING,
    nthread=8,
    **params
    )

In [66]:
final_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=-9999, n_estimators=100, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [67]:
y_pred_train = final_model.predict(X_train)
y_prob_train = final_model.predict_proba(X_train)[:,1]

In [68]:
y_pred_test = final_model.predict(X_test)
y_prob_test = final_model.predict_proba(X_test)[:,1]

In [69]:
roc_auc_score(y_test, y_prob_test), roc_auc_score(y_train, y_prob_train)

(0.8901553362068626, 0.9203028581243273)

In [ ]:
(0.8901941061759845, 0.9386534084249205)  #5

In [ ]:
(0.8901553362068626, 0.9203028581243273) #4

In [ ]:
(0.8896688507329076, 0.9068574909384938) #3

In [ ]:
(0.8879588067229542, 0.8971857360901431) #2

In [70]:
y_pred_final = final_model.predict(X_final)
y_prob_final = final_model.predict_proba(X_final)[:,1]

In [71]:
df_result = pd.DataFrame(y_prob_final, columns=["Predicted"])

In [72]:
df_result["Id"] = df_test["id"].values

In [73]:
df_result[["Id", "Predicted"]]

,Id,Predicted
0,14061,0.080889
1,16467,0.106882
2,25725,0.881758
3,9100,0.854454
4,32597,0.638906
...,...,...
9240,31430,0.870307
9241,3876,0.679088
9242,20710,0.690683
9243,26015,0.741808


In [74]:
df_result[["Id", "Predicted"]].to_csv("../data/result/result_03_02_submission2.csv", index=False)